# Mise en situation - Détection cancer du sein 

Vous travaillez pour un clinique qui souhaite développer un modèle permettant de prédire la présence de cellules cancéreuse à partir des caractéristiques de cellules prélevées sur le sein à l'aide d'une biopsie à l'aiguille fine.  

Pour cette mission, vous disposez du fichier de données `breast_cancer.csv`, de la documentation de ces données (https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data?resource=download) et du code d'un consultant avant vous, qui a eu le temps d'entraîner un premier modèle de prédiction.  

Cependant, ce consultant n'a pas fait de travail préliminaire d'exploration des données et le client pense que le modèle pourrait être amélioré. Votre mission est donc :  

- De présenter de manière extensive les données à votre client, par des statistiques descriptives et des visualisations adaptées.  
- De proposer de nouveaux modèles de prédiction et de présenter les métriques d'évaluation adaptées pour justifier ou non des meilleurs résultats que le précédent.  

# 1.  Analyse exploratoire des données  

Pour cette partie, vous êtes libres des choix que vous ferez. Il faut cependant que votre notebook contienne :  
- Affichage de quelques lignes du dataframe pour montrer sa structure.  
- Présentation des différentes variables de la base et de leur type.  
- Statistiques descriptives sur chacune des variables en fonction de leur type.  
- Présenter des statistiques par diagnostic (bénin/malin) pour les variables, permettant de mettre en évidence l'aspect plus ou moins discriminant de certaines variables.  
- Illustrer à l'aide de graphiques les variables les plus discriminantes en fonction du diagnostic (scatterplot avec différentes couleurs).  

# 2. Évaluation d'un modèle  

Pour cette partie, vous disposez du code du consultant précédent sur lequel vous pouvez vous appuyer. Votre mission :  
- Expliquez les étapes du code réalisé :  
        - En quoi consiste la standardisation des données? Pourquoi est-elle importante ici?   
        - Expliquez à quoi sert le fait de séparer les données en entraînement et en test.  
- Expliquez simplement comment fonctionne les arbres de décision.
- Quelles sont les limites de la métrique choisie pour évaluer le modèle? Pour le même modèle, sortez d'autres métriques d'évaluation en les commentant (précision, recall, F1 score, matrice de confusion...)

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import pandas as pd

breast_cancer = pd.read_csv("data/breast_cancer.csv")

# On sépare les features du target 
features, target = breast_cancer.loc[:, 'radius_mean':'fractal_dimension_worst'], breast_cancer["diagnosis"]

# On split notre dataframe en 2
feat_train, feat_test, target_train, target_test = train_test_split(
    features, target,
    test_size = 0.4)

# On standardise  
scaler = StandardScaler()
feat_train = scaler.fit_transform(feat_train)
feat_test = scaler.transform(feat_test)

# On initalise le modèle 
tree_classif = tree.DecisionTreeClassifier()

# On l'entraîne sur nos données d'entraînement
tree_classif.fit(feat_train, target_train)

# On l'utilise pour prédire sur les données de test
predict_class = tree_classif.predict(feat_test)

# On calcule l'accuracy du modèle  
acc = accuracy_score(predict_class, target_test)
print(acc)

0.9385964912280702


# 3. Entraînement d'un modèle concurrent   
En suivant la même logique, entraînez un modèle de classification avec la méthode des k plus proches voisins (ou k-nn). Expliquez comment il fonctionne. Sélectionnez le nombre de voisins qui vous donne le meilleur F1-score (en testant de 2 à 10 voisins) et comparez les performances de ce modèle avec le modèle précédent selon les différentes métriques.  

__Bonus__ : Si vous avez le temps, entraînez un autre modèle de classification issu de `sklearn`, décrivez-le et présentez ses performances.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors, metrics
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')


import pandas as pd

breast_cancer = pd.read_csv("data/breast_cancer.csv")

# On sépare les features du target 
features, target = breast_cancer.loc[:, 'radius_mean':'fractal_dimension_worst'], breast_cancer["diagnosis"]

# On split notre dataframe en 2
feat_train, feat_test, target_train, target_test = train_test_split(
    features, target,
    test_size = 0.4)

# On standardise
scaler = StandardScaler()
feat_train = scaler.fit_transform(feat_train)
feat_test = scaler.transform(feat_test)


# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[2, 3, 4, 5, 6, 7, 8, 9, 10]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,
    scoring=score)   # score à optimiser

clf.fit(feat_train, target_train)


# Afficher les performances correspondantes
print("Résultats de la validation croisée:")
for mean, std, params in zip(clf.cvresults['mean_test_score'], # score moyen
                                       clf.cvresults['std_test_score'],
                                       clf.cvresults['params']):

    print("{} = {:.3f} (+/-{:.03f}) for {}".format(score,mean,std*2,params))